#### retreving all dataset.tt files and their commands 

In [2]:
import glob
path = '/Users/madinaabdrakhmanova/Documents/text_commands_gen/thingpedia-common-devices-master/'
file_paths = [f for f in glob.glob(path + '**/dataset.tt', recursive=True)]

In [3]:
def isEnum(contents, ind):
    l = contents[ind:].find("Enum(")
    r = contents[ind:].find(")")
    enum_strs = []
    if (int(l) != -1 and int(r) != -1 and r > l): 
        enum_strs = contents[ind:][l+len("Enum("):r]
        enum_strs = enum_strs.split(",")
    return enum_strs

In [4]:
def findEnum(action_str):
    ind = action_str.find('$')
    res = ''
    while (ind < len(action_str) and action_str[ind] != ' ' and action_str[ind] != '"'):
        res = res + action_str[ind]
        ind += 1
    return res

In [5]:
def subst_enum(action_strs, enum_strs):
    result = []
    for action_str in action_strs:
        if ('$' in action_str and len(enum_strs) > 0):
            '''
            u = action_str.find('$')
            v = action_str.find('}')
            rep = action_str[u:v+1]
            '''
            rep = findEnum(action_str)
            for enum_str in enum_strs:
                #print("before\n", action_str)
                action_str_rep = action_str.replace(rep,enum_str)
                #print("after\n", action_str_rep)
                if ('$' not in action_str_rep): result.append(action_str_rep.strip().replace('"',''))
        elif ('$' not in action_str):
            result.append(action_str.strip().replace('"',''))
    return result

In [6]:
commands_action = []
for file_path in file_paths:
    f=open(file_path, "r")
    contents =f.read()
    action_start_idx_lst = [i for i in range(len(contents)) if contents.startswith('action', i)]
    variable_set = []
    for i in action_start_idx_lst:   
        enum_strs = isEnum(contents, i)
        j = contents[i:].find("#_[utterances=[")
        k = contents[i:].find("]]")
        action_strs = contents[i:][j+len("#_[utterances=["):k]
        action_strs = action_strs.split(',')
        #print('action_strs\n', action_strs)
        result = []
        if(len(enum_strs) > 0): result = subst_enum(action_strs, enum_strs)
        else: result = [action_str.strip().replace('"','') for action_str in action_strs if (action_str.strip().replace('"','') and ('$' not in action_str))]
        commands_action = commands_action + result

In [29]:
print("number of all action commands: ",len(commands_action))
print("number of all unique action commands: ",len(set(commands_action)))
print("here is the first action command: ", commands_action[0])
[l for l in commands_action if (len(l.split())<2)]

number of all action commands:  350
number of all unique action commands:  331
here is the first action command:  tweet a picture


[]

In [42]:
import re
[c for c in commands_action if (len(re.findall(r'\d+',c))>0)]

[]

In [8]:
commands_query = []
for file_path in file_paths:
    f=open(file_path, "r")
    contents =f.read()
    action_start_idx_lst = [i for i in range(len(contents)) if contents.startswith('query', i)]
    for i in action_start_idx_lst:
        enum_strs = isEnum(contents, i)
        j = contents[i:].find("#_[utterances=[")
        k = contents[i:].find("]]")
        action_strs = contents[i:][j+len("#_[utterances=["):k]
        action_strs = action_strs.split(',')
        #print('action_strs\n', action_strs)
        if (len(enum_strs) > 0): result = subst_enum(action_strs, enum_strs)
        else: result = [action_str.strip().replace('"','') for action_str in action_strs if (action_str.strip().replace('"','') and ('$' not in action_str))]
        #print('result\n', result)
        commands_query = commands_query + result

In [15]:
print("number of all query commands: ",len(commands_query))
print("number of all unique query commands: ",len(set(commands_query)))
print("here is the first query command: ", commands_query[0])


number of all query commands:  753
number of all unique query commands:  729
here is the first query command:  posts on phd comics
posts on phd comics


In [31]:
commands_query_f = [l for l in commands_query if (len(l)>2)]

In [34]:
commands_query_f = [l for l in commands_query_f if (len(l.split())>1)]

In [53]:
new_commands_query_f = []
for c in commands_query_f:
    if "matrix67" in c:
        c = c.replace('matrix67','matrix sixty seven')
        print(c)
    if "cs294" in c:
        c = c.replace('cs294','cs two nine four ')
        print(c)
        
    new_commands_query_f.append(c)

posts on matrix sixty seven
latest posts on matrix sixty seven
posts on matrix sixty seven blog
cs two nine four s class slides
cs two nine four s slides
cs two nine four s class materials
cs two nine four w class slides
cs two nine four w slides


In [54]:
import re
[c for c in new_commands_query_f if (len(re.findall(r'\d+',c))>0)]


[]

In [55]:
all_commands =  list(set(commands_action))+ list(set(new_commands_query_f))
print(len(all_commands))

1050


#### saving as .csv file

In [57]:
import pandas as pd
df = pd.DataFrame()
df["command_name"] = all_commands
df["command_id"] = range(1,len(all_commands)+1)
df["num_chars"] = [len(command) for command in all_commands]
df = df.set_index('command_id')
df.to_csv('csvs/commands_v1.csv')
df.head(10)

,command_name,num_chars
command_id,,
1,alert me by flashing the lights,31
2,open my window for me,21
3,increase volume on my speaker,29
4,order me a macchiato,20
5,put the curtains down,21
6,order me a americano,20
7,close my window,15
8,upload a picture to Imgur,25
9,open my curtains,16


#### saving as .mat file

In [48]:
import scipy.io as sio
sio.savemat('commands.mat', {"command_name": all_commands, 
                             "command_id" : range(len(all_commands)),
                             "num_chars": [len(command) for command in all_commands]}) 